https://blog.csdn.net/baoyan2015/article/details/135777614

In [1]:
import os
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'
import warnings
warnings.filterwarnings("ignore")

from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig, LlamaTokenizer, LlamaForCausalLM
import torch

# model_name_or_id = "AI4Chem/ChemLLM-7B-Chat" 
# model_name_or_id = "AI4Chem/ChemLLM-20B-Chat-DPO"
# model_name_or_id = "AI4Chem/ChemLLM-20B-Chat-SFT"
model_name_or_id = "AI4Chem/ChemLLM-7B-Chat-1_5-SFT"

# model_name_or_id = "X-LANCE/ChemDFM-13B-v1.0"

In [2]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, trust_remote_code=True,
                                             device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id,trust_remote_code=True)

# model = LlamaForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, trust_remote_code=True,
#                                             device_map="auto")
# tokenizer = LlamaTokenizer.from_pretrained(model_name_or_id, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

当使用大型语言模型（如 GPT-2、GPT-3、BERT 等）的 generate 方法直接产生文本时，通常返回的是文本的 token ID 序列。  
为了获得每个 token 的生成概率，你需要使用模型的 logits 输出。logits 是模型在softmax层之前的输出，表示模型对每个可能的下一个 token 的置信度。通过对这些 logits 应用softmax函数，可以得到概率分布。

In [3]:
# encoder input text
input_text = "The quick brown fox"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# generate output text and get logits
outputs = model.generate(input_ids,
                         max_length=20,
                         output_scores=True, # return every generated token's logits
                         return_dict_in_generate=True)

# extract logits
logits = outputs.scores

# compute probability
probs = [torch.softmax(log, dim=-1) for log in logits]

# get token ID of the generated output and corresponding probabilities
generated_ids = outputs.sequences

for i, token_id in enumerate(generated_ids[0][ len(input_ids[0]): ]):
    token_prob = probs[i][0, token_id].item()
    print(f"Token ID: {token_id}, Probability: {token_prob}")

2024-07-22 09:19:59.133534: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-22 09:19:59.993451: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Token ID: 26444, Probability: 0.46754440665245056
Token ID: 1053, Probability: 0.9870383739471436
Token ID: 410, Probability: 0.9968327879905701
Token ID: 15810, Probability: 0.9913532137870789
Token ID: 5718, Probability: 0.9688257575035095
Token ID: 281, Probability: 0.5558463931083679
Token ID: 707, Probability: 0.19871832430362701
Token ID: 4131, Probability: 0.45601770281791687
Token ID: 14018, Probability: 0.959482729434967
Token ID: 38648, Probability: 0.8549602031707764
Token ID: 26444, Probability: 0.8256170749664307
Token ID: 1053, Probability: 0.9749144911766052
Token ID: 410, Probability: 0.9972091317176819
Token ID: 15810, Probability: 0.9762767553329468
Token ID: 5718, Probability: 0.9929332733154297


In [14]:
prob = torch.softmax(logits[0], dim=-1)
idx = prob.argmax(-1) # 使用argmax函数找到概率分布中最大值的索引。-1参数表示在最后一个维度上进行操作，即在每个样本的概率分布中找到最大值的索引。这通常用于确定模型预测的类别。
prob.shape, idx

(torch.Size([1, 92544]), tensor([26444]))

In [19]:
# result = [prob[i][y] if 'yes'in tokenizer.convert_ids_to_tokens(int(y.detach())).lower() else 1-prob[i][y] for i, y in enumerate(idx)]
for i, y in enumerate(idx):
    print(i, y)
    print(prob[i][y])
    result = []
    print(tokenizer.convert_ids_to_tokens(int(y.detach())).lower())
    if 'yes'in tokenizer.convert_ids_to_tokens(int(y.detach())).lower():
        result.append(prob[i][y])
    else:
        result.append(1-prob[i][y])

result

0 tensor(26444)
tensor(0.4675)
▁jumped


[tensor(0.5325)]

In [6]:
import deepchem
import deepchem.molnet
tasks, datasets, transformers = deepchem.molnet.load_clintox(featurizer=deepchem.feat.RawFeaturizer(True),
                                                             splitter='scaffold', reload=True,
                                                             data_dir='./data/clintox_data',
                                                             save_dir='./data/clintox_datasets')

train_dataset, valid_dataset, test_dataset = datasets
for i in range(5):
    print(f"X: {test_dataset.X[i]} y: {test_dataset.y[i]} id: {test_dataset.ids[i]}")

[09:27:04] Explicit valence for atom # 0 N, 5, is greater than permitted
Failed to featurize datapoint 7, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True, bool includeAtomMaps=True)
[09:27:04] Can't kekulize mol.  Unkekulized atoms: 9
Failed to featurize datapoint 302, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(NoneType)
did not match C++ signature:
    CanonicalRankAtoms(RDKit::ROMol mol, bool breakTies=True, bool includeChirality=True, bool includeIsotopes=True, bool includeAtomMaps=True)
[09:27:05] Can't kekulize mol.  Unkekulized atoms: 4
Failed to featurize datapoint 1219, None. Appending empty array
Exception message: Python argument types in
    rdkit.Chem.rdmolfiles.CanonicalRankAtoms(

X: C[NH+]1C[C@H](C(=O)N[C@]2(C)O[C@@]3(O)[C@@H]4CCCN4C(=O)[C@H](Cc4ccccc4)N3C2=O)C[C@@H]2c3cccc4[nH]cc(c34)C[C@H]21 y: [1. 0.] id: C[C@@]1(C(=O)N2[C@H](C(=O)N3CCC[C@H]3[C@@]2(O1)O)Cc4ccccc4)NC(=O)[C@@H]5C[C@@H]6c7cccc8c7c(c[nH]8)C[C@H]6[NH+](C5)C
X: C[C@]1(Cn2ccnn2)[C@H](C(=O)[O-])N2C(=O)C[C@H]2S1(=O)=O y: [1. 0.] id: C[C@@]1([C@@H](N2[C@H](S1(=O)=O)CC2=O)C(=O)[O-])Cn3ccnn3
X: C[NH+]1CCC[C@@H]1CCO[C@](C)(c1ccccc1)c1ccc(Cl)cc1 y: [1. 0.] id: C[C@@](c1ccccc1)(c2ccc(cc2)Cl)OCC[C@H]3CCC[NH+]3C
X: Nc1nc(NC2CC2)c2ncn([C@H]3C=C[C@@H](CO)C3)c2n1 y: [1. 0.] id: c1nc2c(nc(nc2n1[C@@H]3C[C@@H](C=C3)CO)N)NC4CC4
X: OC[C@H]1O[C@@H](n2cnc3c2NC=[NH+]C[C@H]3O)C[C@@H]1O y: [1. 0.] id: c1nc2c(n1[C@H]3C[C@@H]([C@H](O3)CO)O)NC=[NH+]C[C@H]2O


In [28]:
import numpy as np
X = list(test_dataset.ids)
label = list(np.array(i[1]) for i in test_dataset.y)
len(label)

148

In [4]:
for i, token_id in enumerate(generated_ids[0][ len(input_ids[0]): ]):
    print(f"i: {i}, token_id: {token_id}")

i: 0, token_id: 26444
i: 1, token_id: 1053
i: 2, token_id: 410
i: 3, token_id: 15810
i: 4, token_id: 5718
i: 5, token_id: 281
i: 6, token_id: 707
i: 7, token_id: 4131
i: 8, token_id: 14018
i: 9, token_id: 38648
i: 10, token_id: 26444
i: 11, token_id: 1053
i: 12, token_id: 410
i: 13, token_id: 15810
i: 14, token_id: 5718


In [5]:
generated_ids

tensor([[    1,   918,  4131, 14018, 38648, 26444,  1053,   410, 15810,  5718,
           281,   707,  4131, 14018, 38648, 26444,  1053,   410, 15810,  5718]])

In [6]:
input_ids

tensor([[    1,   918,  4131, 14018, 38648]])

In [7]:
generated_ids[0][ len(input_ids[0]): ]

tensor([26444,  1053,   410, 15810,  5718,   281,   707,  4131, 14018, 38648,
        26444,  1053,   410, 15810,  5718])

In [9]:
probs, len(probs)

([tensor([[0.0000e+00, 0.0000e+00, 2.7255e-05,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00]]),
  tensor([[0.0000e+00, 0.0000e+00, 1.7375e-06,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00]]),
  tensor([[0.0000e+00, 0.0000e+00, 1.4404e-07,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00]]),
  tensor([[0.0000e+00, 0.0000e+00, 1.1156e-07,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00]]),
  tensor([[0.0000e+00, 0.0000e+00, 1.7976e-07,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00]]),
  tensor([[0.0000, 0.0000, 0.0002,  ..., 0.0000, 0.0000, 0.0000]]),
  tensor([[1.3865e-31, 1.1124e-28, 5.5816e-04,  ..., 7.6454e-29, 6.9717e-32,
           0.0000e+00]]),
  tensor([[1.0920e-38, 8.5892e-38, 6.7213e-06,  ..., 8.8486e-35, 1.6047e-37,
           0.0000e+00]]),
  tensor([[0.0000e+00, 0.0000e+00, 7.5697e-06,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00]]),
  tensor([[0.0000e+00, 0.0000e+00, 1.1121e-05,  ..., 0.0000e+00, 0.0000e+00,
           0.0000e+00]]),
  ten

In [11]:
probs[0], probs[0].shape

(tensor([[0.0000e+00, 0.0000e+00, 2.7255e-05,  ..., 0.0000e+00, 0.0000e+00,
          0.0000e+00]]),
 torch.Size([1, 92544]))

In [12]:
probs[0][0, 7560].item() # 7560为ChemLLM词表中Yes对应的id, 查看输出yes的概率

8.674778229078584e-08

In [13]:
probs[0][0, 26444]

tensor(0.4675)

从 Token ID 获取对应的文本  

若要从已知的 token ID 获取对应的文本，你需要使用加载的 tokenizer 对象的 decode 方法。以下是如何操作的步骤：
1. 加载预训练的 tokenizer。
2. 使用 decode 方法将 token ID 转换为文本。

In [24]:
token_ids = generated_ids[0][ len(input_ids[0]): ]

# use `decode` method to convert token ID to text
text = tokenizer.decode(token_ids)

print(text)

 jumped over the lazy dog. The quick brown fox jumped over the lazy dog


In [3]:
token_ids = [7560, 2458]

# use `decode` method to convert token ID to text
text = tokenizer.decode(token_ids)

print(text)

2024-07-21 16:43:00.721890: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-21 16:43:01.661077: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


 Yes No


In [6]:
# encoder input text
input_text = "Yes"
input_ids = tokenizer.encode(input_text, return_tensors='pt', add_special_tokens=False)
input_ids[0]

tensor([9583])

In [3]:
token_ids = [7560, 2458, 9583, 2917, 2]

# use `decode` method to convert token ID to text
text = tokenizer.decode(token_ids)

print(text)

2024-07-29 15:03:58.915601: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-29 15:03:59.761033: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


  Yes NoYesNo</s>


In [3]:
# ChemDFM
token_ids = [3869, 4874, 1939, 694, 8241, 3782]

# use `decode` method to convert token ID to text
text = tokenizer.decode(token_ids)

print(text)

2024-07-23 09:43:03.340089: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-23 09:43:04.160541: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Yes yes No noYesNo
